## 1단계: 필수 패키지 설치

In [ ]:
!pip install --upgrade azure-ai-evaluation
!pip install pandas numpy matplotlib seaborn

## 2단계: 라이브러리 임포트 및 데이터 로드

In [1]:
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Microsoft Foundry SDK - Groundedness Evaluator
try:
    from azure.ai.evaluation import GroundednessEvaluator
    print("✓ Azure AI Evaluation 임포트 성공")
except ImportError:
    print("⚠ Azure AI Evaluation 설치 필요: pip install azure-ai-evaluation")

# 데이터 로드
# with open('data/filtered_qa.json', 'r', encoding='utf-8') as f:
with open('data/sample_qa.json', 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

df = pd.DataFrame(qa_data)

print(f"\n✓ 데이터 로드 완료: {len(df)} 개 Q&A 쌍")
print(f"\n에이전트별 분포:")
print(df['agent'].value_counts())

✓ Azure AI Evaluation 임포트 성공

✓ 데이터 로드 완료: 34 개 Q&A 쌍

에이전트별 분포:
agent
product-agent          10
hallucination-agent     7
review-agent            6
policy-agent            6
rag-fail-agent          5
Name: count, dtype: int64


## 3단계: Groundedness Evaluator 초기화

In [ ]:
# ============================================================
# model_config 설정
# ============================================================
model_config = {
    "azure_endpoint": "https://your-endpoint.openai.azure.com/",
    "api_key": "your-api-key",
    "azure_deployment": "gpt-4",
    "api_version": "2024-12-01-preview"
}

# Groundedness Evaluator 초기화 (model_config 필수)
try:
    groundedness_evaluator = GroundednessEvaluator(model_config=model_config)
    
    print("\n✓ Groundedness Evaluator 초기화 성공")
    print("\n📋 평가 입력:")
    print("  - query: 사용자 질문 (선택)")
    print("  - response: 평가할 응답 (필수)")
    print("  - context: 참조 컨텍스트 (필수)")
    print("\n📋 출력 메트릭:")
    print("  - groundedness: 근거성 점수 (1-5)")
    print("    * 5: 완전히 컨텍스트에 기반")
    print("    * 3: 부분적으로 기반")
    print("    * 1: 근거 없음/할루시네이션")
except Exception as e:
    print(f"\n⚠ Groundedness Evaluator 초기화 실패: {str(e)[:200]}")


✓ Groundedness Evaluator 초기화 성공

📋 평가 입력:
  - query: 사용자 질문 (선택)
  - response: 평가할 응답 (필수)
  - context: 참조 컨텍스트 (필수)

📋 출력 메트릭:
  - groundedness: 근거성 점수 (1-5)
    * 5: 완전히 컨텍스트에 기반
    * 3: 부분적으로 기반
    * 1: 근거 없음/할루시네이션


## 4단계: 전체 데이터에 대한 Groundedness Evaluator 실행

In [3]:
# 평가를 위한 데이터 준비
groundedness_eval_data = []
for idx, row in df.iterrows():
    groundedness_eval_data.append({
        'query': row['question'],
        'context': row.get('context', row['question']),
        'response': row['response'],
        'agent': row['agent']
    })

print("=== 전체 데이터 Groundedness 평가 시작 ===")
print(f"대상: {len(groundedness_eval_data)} 개 Q&A 쌍\n")

groundedness_results = []

for i, item in enumerate(groundedness_eval_data):
    try:
        result = groundedness_evaluator(
            query=item['query'],
            context=item['context'],
            response=item['response']
        )
        result['agent'] = item['agent']
        result['context_preview'] = item['context'][:50] + '...' if len(item['context']) > 50 else item['context']
        groundedness_results.append(result)
        
        if (i + 1) % 10 == 0:
            print(f"  [{i + 1}/{len(groundedness_eval_data)}] 완료")
    except Exception as e:
        print(f"  [{i + 1}/{len(groundedness_eval_data)}] 오류: {str(e)[:60]}")

if groundedness_results:
    groundedness_df = pd.DataFrame(groundedness_results)
    
    print(f"\n✓ 평가 완료: {len(groundedness_results)}개")
    print(f"평균 Groundedness 점수: {groundedness_df['groundedness'].mean():.2f}/5\n")
    
    # 전체 결과 표시
    print("=== 평가 결과 ===")
    from IPython.display import display
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    display(groundedness_df)
else:
    print("\n⚠ 평가 결과가 없습니다. Azure OpenAI 연결을 확인하세요.")

=== 전체 데이터 Groundedness 평가 시작 ===
대상: 34 개 Q&A 쌍

  [10/34] 완료
  [20/34] 완료
  [30/34] 완료

✓ 평가 완료: 34개
평균 Groundedness 점수: 3.85/5

=== 평가 결과 ===


,groundedness,gpt_groundedness,groundedness_reason,groundedness_result,groundedness_threshold,groundedness_prompt_tokens,groundedness_completion_tokens,groundedness_total_tokens,groundedness_finish_reason,groundedness_model,groundedness_sample_input,groundedness_sample_output,agent,context_preview
0,5.0,5.0,The response fully and accurately answers the ...,pass,3,1428,197,1625,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",product-agent,"갤럭시 S24 사양: 배터리 4,000mAh, 25W 유선 고속 충전, 15W 무선..."
1,5.0,5.0,"The response is fully correct and complete, di...",pass,3,1427,174,1601,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",product-agent,"노트북 A1: 기본 RAM 16GB DDR5, 최대 32GB 지원, 2개 메모리 슬..."
2,5.0,5.0,The response fully and accurately answers the ...,pass,3,1422,145,1567,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",product-agent,"무선 이어폰 X3: 이어버드 8시간, 케이스 포함 28시간, 급속 충전 10분=2시..."
3,4.0,4.0,The response is relevant and partially correct...,pass,3,1387,181,1568,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",product-agent,"스마트워치 W2: IP68 등급, 5ATM 방수, 최대 50m 수심"
4,4.0,4.0,The response is relevant to the query but does...,pass,3,1390,173,1563,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",product-agent,"태블릿 T5: 11인치 AMOLED 디스플레이, 2560x1600 해상도, 120H..."
5,1.0,1.0,The response does not address the question abo...,fail,3,1395,170,1565,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",product-agent,"카메라 C1: 풀프레임 센서 (36mm x 24mm), 4500만 화소"
6,1.0,1.0,The response does not answer the question abou...,fail,3,1395,151,1546,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",product-agent,"청소 로봇 R7: 먼지통 600ml, 자동 먼지 배출 스테이션 2.5L"
7,5.0,5.0,"The response is fully grounded in the context,...",pass,3,1463,171,1634,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",review-agent,"노트북 리뷰 통계: 평균 4.5/5점, 긍정: 속도, 멀티태스킹, 조용함, 부정: 배터리"
8,5.0,5.0,The response accurately and completely reflect...,pass,3,1439,177,1616,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",review-agent,"이어폰 X3 리뷰: 음질 만족도 87%, 저음 우수, 노이즈 캔슬링 우수, 고음 날..."
9,4.0,4.0,The response is accurate in stating that revie...,pass,3,1391,229,1620,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""\\...","[{""role"": ""assistant"", ""content"": ""<S0>Let's t...",review-agent,"스마트워치 W2 리뷰: 평균 4.2/5점, 배터리 수명 우수, 건강 추적 정확, 앱..."


## 5단계: 할루시네이션 분석

In [4]:
# 할루시네이션 분석 (샘플 결과가 있을 경우)
if groundedness_results:
    print("=== 할루시네이션 분석 ===")
    print(f"\n평가된 샘플: {len(groundedness_results)}개\n")
    
    for i, row in groundedness_df.iterrows():
        # groundedness 키 확인 (gpt_groundedness 또는 groundedness)
        score = row.get('groundedness', row.get('gpt_groundedness', 0))
        agent = row['agent']
        
        # 점수 범위가 1-5이므로 기준 조정
        if score >= 4:
            status = "✓ 안전"
        elif score >= 3:
            status = "⚠ 부분적 위험"
        else:
            status = "✗ 고위험 (할루시네이션)"
        
        print(f"[{i+1}] {agent}: {score:.2f}/5 {status}")
else:
    print("⚠ 평가 결과가 없습니다. Azure OpenAI 연결을 확인하세요.")

=== 할루시네이션 분석 ===

평가된 샘플: 34개

[1] product-agent: 5.00/5 ✓ 안전
[2] product-agent: 5.00/5 ✓ 안전
[3] product-agent: 5.00/5 ✓ 안전
[4] product-agent: 4.00/5 ✓ 안전
[5] product-agent: 4.00/5 ✓ 안전
[6] product-agent: 1.00/5 ✗ 고위험 (할루시네이션)
[7] product-agent: 1.00/5 ✗ 고위험 (할루시네이션)
[8] review-agent: 5.00/5 ✓ 안전
[9] review-agent: 5.00/5 ✓ 안전
[10] review-agent: 4.00/5 ✓ 안전
[11] review-agent: 4.00/5 ✓ 안전
[12] policy-agent: 5.00/5 ✓ 안전
[13] policy-agent: 5.00/5 ✓ 안전
[14] policy-agent: 4.00/5 ✓ 안전
[15] policy-agent: 4.00/5 ✓ 안전
[16] product-agent: 5.00/5 ✓ 안전
[17] product-agent: 4.00/5 ✓ 안전
[18] product-agent: 5.00/5 ✓ 안전
[19] review-agent: 5.00/5 ✓ 안전
[20] review-agent: 5.00/5 ✓ 안전
[21] policy-agent: 5.00/5 ✓ 안전
[22] policy-agent: 5.00/5 ✓ 안전
[23] hallucination-agent: 2.00/5 ✗ 고위험 (할루시네이션)
[24] hallucination-agent: 1.00/5 ✗ 고위험 (할루시네이션)
[25] hallucination-agent: 2.00/5 ✗ 고위험 (할루시네이션)
[26] hallucination-agent: 2.00/5 ✗ 고위험 (할루시네이션)
[27] hallucination-agent: 2.00/5 ✗ 고위험 (할루시네이션)
[28] hallucination-agent: